# Preparação Dos Dados


In [1]:
import pandas as pd
import numpy as np

file_path = 'data/TUCURUI_DIARIA_1939_2016.xlsx'
data = pd.read_excel(file_path, header=None, names=['Vazao'])

# Define os intervalos de defasagem a serem testados
lag_ranges = [15, 30, 45]

# Loop sobre cada intervalo de defasagem
for lag_range in lag_ranges:
    
    for i in range(lag_range, 0, -1):
        data[f'Entrada {i}'] = data['Vazao'].shift(i)

    # Define a ordem das novas colunas (defasagens de entrada + coluna original 'Vazao')
    novas_colunas = [f'Entrada {i}' for i in range(lag_range, 0, -1)] + ['Vazao']
    data = data[novas_colunas]

    # Criação das colunas de saída defasadas
    for i in range(1, 12):
        data[f'Saida {i}'] = data['Vazao'].shift(-i)

    # escolhi  exibir a primeira e a última coluna
    subset_columns =  [data.columns[0], 'Vazao', data.columns[-1]]
    
    print(f"\nDataframe for lag range {lag_range}:")
    print(data.head(2).loc[:, subset_columns]) # Exibe as primeiras 2 linhas do DataFrame para verificar o resultado



Dataframe for lag range 15:
   Entrada 15  Vazao  Saida 11
0         NaN   2110    2000.0
1         NaN   2089    2000.0

Dataframe for lag range 30:
   Entrada 30  Vazao  Saida 11
0         NaN   2110    2000.0
1         NaN   2089    2000.0

Dataframe for lag range 45:
   Entrada 45  Vazao  Saida 11
0         NaN   2110    2000.0
1         NaN   2089    2000.0
